In [1]:
###############################################################################
### pthon imports                                                           ###
###############################################################################

from os import path
from requests import get
from bs4 import BeautifulSoup
import os
import pandas as pd
import re

In [2]:
###############################################################################
### local imports                                                           ###
###############################################################################

import zachquire as zaq

In [3]:
def find_file(
    file_name='path/file.txt',
    cache=False,
    cache_age=None
    ):

    if cache==False:
        return None
    
    return path.exists(file_name)

In [4]:
def get_soup(
    url='https://github.com/search?o=desc&p=1&q=advent+of+code&s=stars&type=Repositories',
    headers={'User-Agent': 'Nothing suspicious'},
    file_name='soupfiles/gitsearch.txt',
    cache=False,
    cache_age=None,
    slurper='*'
):
    '''
    
    '''
    # if we already have the data, read it locally
    file_found = find_file(file_name=file_name, cache=cache, cache_age=cache_age)
    if file_found:
        with open(file_name) as f:
            return True, BeautifulSoup(f.read())

    # otherwise go fetch the data
    response = get(url, headers=headers)
    soup = BeautifulSoup(response.text)
    slurps = soup.select(slurper)
    if len(slurps) == 0:
        print('***** NO DATA RETURNED *****')
        return False, soup
    
    # save it for next time
    with open(file_name, 'w') as f:
        f.write(str(slurps[0]))
        if len(slurps)>1:
            for slurp in slurps[1:]:
#                 print(str(slurp))
                f.write('\n' + str(slurp))
        
    with open(file_name) as f:    
        soup = BeautifulSoup(f.read())
#     pd.to_csv(slurps, header=None, index=False)

    return True, soup

In [5]:
slurped, soup = get_soup(
    url='https://github.com/search?o=desc&p=2&q=advent+of+code&s=stars&type=Repositories',
    cache=True,
    slurper='.repo-list-item a',
    file_name='soupfiles/gitsearch.txt'
)

In [6]:
def soup_loop_gitsearch(
    df,
    page_beg=1,
    page_end=20,
    url='https://github.com/',
    adder_prepage='search?o=desc&',
    adder_postpage='&q=advent+of+code&s=stars&type=Repositories',
    headers={'User-Agent': 'Nothing suspicious'},
    directory='soupfiles/',
    file_name='gitsearch',
    file_suffix='.txt',
    cache=False,
    slurper='.repo-list-item a',
):
    for page in range(page_beg, page_end+1):
        use_page = str(page)
        use_url = f'{url}{adder_prepage}p={use_page}{adder_postpage}'
        use_file = f'{directory}{file_name}{use_page:>02s}{file_suffix}'
        print(f'use_page: {use_page:>2s}')
        print(f'use_url: {use_url}')
        print(f'use_file: {use_file}')
        slurped, soup = get_soup(
            url = use_url,
            file_name = use_file,
            cache = cache,
            slurper = slurper,
        )
        log_file = use_file if slurped else None
        loopdict = {
            'soup': [soup], 
            'page': [page], 
            'url': [use_url], 
            'file_name': [log_file]
        }
        df = df.append(pd.DataFrame.from_dict(loopdict), ignore_index=True)
    return df
        

In [7]:
soup_df = pd.DataFrame([], columns=['soup','page','url','file_name'])
# df
soup_df = soup_loop_gitsearch(soup_df, cache=True, page_beg=1, page_end=20)

use_page:  1
use_url: https://github.com/search?o=desc&p=1&q=advent+of+code&s=stars&type=Repositories
use_file: soupfiles/gitsearch01.txt
use_page:  2
use_url: https://github.com/search?o=desc&p=2&q=advent+of+code&s=stars&type=Repositories
use_file: soupfiles/gitsearch02.txt
use_page:  3
use_url: https://github.com/search?o=desc&p=3&q=advent+of+code&s=stars&type=Repositories
use_file: soupfiles/gitsearch03.txt
use_page:  4
use_url: https://github.com/search?o=desc&p=4&q=advent+of+code&s=stars&type=Repositories
use_file: soupfiles/gitsearch04.txt
use_page:  5
use_url: https://github.com/search?o=desc&p=5&q=advent+of+code&s=stars&type=Repositories
use_file: soupfiles/gitsearch05.txt
use_page:  6
use_url: https://github.com/search?o=desc&p=6&q=advent+of+code&s=stars&type=Repositories
use_file: soupfiles/gitsearch06.txt
use_page:  7
use_url: https://github.com/search?o=desc&p=7&q=advent+of+code&s=stars&type=Repositories
use_file: soupfiles/gitsearch07.txt
use_page:  8
use_url: https://gith

In [8]:
soup_df

,soup,page,url,file_name
0,"<html><body><a class=""v-align-middle"" data-hyd...",1,https://github.com/search?o=desc&p=1&q=advent+...,soupfiles/gitsearch01.txt
1,"<html><body><a class=""v-align-middle"" data-hyd...",2,https://github.com/search?o=desc&p=2&q=advent+...,soupfiles/gitsearch02.txt
2,"<html><body><a class=""v-align-middle"" data-hyd...",3,https://github.com/search?o=desc&p=3&q=advent+...,soupfiles/gitsearch03.txt
3,"<html><body><a class=""v-align-middle"" data-hyd...",4,https://github.com/search?o=desc&p=4&q=advent+...,soupfiles/gitsearch04.txt
4,"<html><body><a class=""v-align-middle"" data-hyd...",5,https://github.com/search?o=desc&p=5&q=advent+...,soupfiles/gitsearch05.txt
5,"<html><body><a class=""v-align-middle"" data-hyd...",6,https://github.com/search?o=desc&p=6&q=advent+...,soupfiles/gitsearch06.txt
6,"<html><body><a class=""v-align-middle"" data-hyd...",7,https://github.com/search?o=desc&p=7&q=advent+...,soupfiles/gitsearch07.txt
7,"<html><body><a class=""v-align-middle"" data-hyd...",8,https://github.com/search?o=desc&p=8&q=advent+...,soupfiles/gitsearch08.txt
8,"<html><body><a class=""v-align-middle"" data-hyd...",9,https://github.com/search?o=desc&p=9&q=advent+...,soupfiles/gitsearch09.txt
9,"<html><body><a class=""v-align-middle"" data-hyd...",10,https://github.com/search?o=desc&p=10&q=advent...,soupfiles/gitsearch10.txt


In [9]:
print(soup_df.soup[19])

<html><body><a class="v-align-middle" data-hydro-click='{"event_type":"search_result.click","payload":{"page_number":20,"per_page":10,"query":"advent of code","result_position":1,"click_id":47785268,"result":{"id":47785268,"global_relay_id":"MDEwOlJlcG9zaXRvcnk0Nzc4NTI2OA==","model_name":"Repository","url":"https://github.com/kodsnack/advent_of_code_2015"},"client_id":null,"originating_request_id":"C7F4:2C7D:5EBEA74:9ED209C:5DF038B4","originating_url":"https://github.com/search?o=desc&amp;p=20&amp;q=advent+of+code&amp;s=stars&amp;type=Repositories","referrer":null,"user_id":null}}' data-hydro-click-hmac="9da75b1cde3ca6cefec42158aacd558eb5da021c451d01da62dd3d92bfe514b2" href="/kodsnack/advent_of_code_2015">kodsnack/<em>advent</em>_<em>of</em>_<em>code</em>_2015</a>
<a href="http://adventofcode.com" rel="nofollow">http://adventofcode.com</a>
<a class="muted-link" href="/kodsnack/advent_of_code_2015/stargazers">
<svg aria-label="star" class="octicon octicon-star" height="16" role="img" ve

In [10]:
re_url = re.compile(r'"url"\:"(.+?)"')
urllist = re_url.findall(str(soup_df.soup[0]))

urllist

['https://github.com/Bogdanp/awesome-advent-of-code',
 'https://github.com/BurntSushi/advent-of-code',
 'https://github.com/Lysxia/advent-of-coq-2018',
 'https://github.com/fogleman/AdventOfCode2018',
 'https://github.com/mstksg/advent-of-code-2018',
 'https://github.com/Voltara/advent2018-fast',
 'https://github.com/wimglenn/advent-of-code-data',
 'https://github.com/ChrisPenner/Advent-Of-Code-Polyglot',
 'https://github.com/jaksi/advent-of-other-peoples-code',
 'https://github.com/molyswu/hand_detection']

In [11]:
def get_repos_from_gitsearch(df, column='soup', reg_text=r'"url"\:"(.+?)"'):
    git_urls = []
    re_url = re.compile(reg_text)
    for soup in soup_df.soup:
        git_urls.extend(re_url.findall(str(soup)))
    return list(set(git_urls))
        

In [12]:
url_list = get_repos_from_gitsearch(soup_df)

In [13]:
len(url_list)

197

In [14]:
remove_repos = [
    'Bogdanp/awesome-advent-of-code',
    'scy/advent-of-code',
    'agrison/advent-of-code-2019/',
    'ephemient/aoc2019/tree/hs',
    'notviri/aoc2019',
    'visini/aoc2019',
]

In [15]:
# souprepo = get_soup(
#     url = 'https://github.com/mfikes/advent-of-code',
#     file_name = 'soupfiles/mfikes_advent-of-code.html',
#     slurper='html',
#     cache = False
# )


In [16]:
# souprepo = get_soup(
#     url = 'https://github.com/Diggsey/aoc2018',
#     file_name = 'soupfiles/diggsey_aoc2018.html',
#     slurper='html',
#     cache = False
# )

In [17]:
# souprepo = get_soup(
#     url = 'https://github.com/mhanberg/advent-of-code-elixir-starter',
#     file_name = 'soupfiles/mhanberg_advent-of-code-elixir-starter.html',
#     slurper='html',
#     cache = False
# )

In [18]:
# Library imports for sending request, parsing HTML, and pandas
from requests import get
from bs4 import BeautifulSoup
import pandas as pd
# Example list of URLs from GitHub
urls = [
    "https://github.com/mfikes/advent-of-code",
    "https://github.com/ryanorsinger/intro-to-machine-learning-workshop",
    'https://github.com/Diggsey/aoc2018',
    "https://github.com/kjeliasen/AdventOfCode",
    'https://github.com/mhanberg/advent-of-code-elixir-starter'
]
output = []
for url in urls:
    request = get(url)
    soup = BeautifulSoup(request.text)
    title = url.split("/")[-1]
    author = url.split("/")[-2]
    language_info = soup.select_one(".repository-lang-stats-numbers")
    languages = [language.get_text() for language in language_info.select(".lang")]
    percentages = [percentage.get_text() for percentage in language_info.select(".percent")]
    readme = soup.select_one("#readme article")
    repo_data = {}
    repo_data["author"] = author
    repo_data["title"] = title
    repo_data["readme"] = readme
    repo_data["languages"] = languages
    repo_data["language_percentages"] = percentages
    output.append(repo_data)
# df = pd.DataFrame(output)
# df.to_csv("soupfiles/all_repo_data.csv")

In [19]:
# df

In [20]:
re_repo = re.compile(r'https\://github.com/(.+?)$')
repo_list = []
repo_check = [re_repo.findall(url)[0] for url in url_list]
repo_list.extend(repo_check)
repo_list.sort()
repo_list

['1Computer1/advent',
 'AlexAegis/advent-of-code',
 'AlexeSimon/adventofcode',
 'BartMassey/advent-of-code-2015',
 'BartMassey/advent-of-code-2016',
 'BartoszMilewski/AdventOfCode18',
 'BartoszMilewski/AoC2019',
 'BartoszMilewski/AofC2017',
 'BenoitZugmeyer/RustyAdventOfCode',
 'Bogdanp/awesome-advent-of-code',
 'BurntSushi/advent-of-code',
 'CameronAavik/AdventOfCode',
 'ChrisPenner/Advent-Of-Code-Polyglot',
 'ChrisPenner/advent-of-code-haskell',
 'CloudCoders/AdventOfCode2017',
 'Dementophobia/advent-of-code-2019',
 'Diggsey/aoc2018',
 'FlorianCassayre/AdventOfCode-2017',
 'FlorianCassayre/AdventOfCode-2018',
 'Harleqin/advent-of-code-2017',
 'IdrisTheDragon/AdventOfCode2018',
 'JEG2/advent_of_code_2016',
 'JEG2/advent_of_code_2018',
 'Janiczek/advent-of-code',
 'KrzaQ/AdventOfCode2015',
 'KrzaQ/AdventOfCode2017',
 'KrzaQ/AdventOfCode2018',
 'KrzaQ/AdventOfCode2019',
 'Larkenx/AdventOfCode2018',
 'Lokeh/advent-2017',
 'Lokeh/advent-2019',
 'Lysxia/advent-of-coq-2018',
 'M0nica/advent

In [21]:
filepath = "datafiles/xtrafile.json"
zaq.scrape_github_data(repos=['Bogdanp/awesome-advent-of-code'], filepath=filepath)
git_xtra = pd.read_json(filepath).rename(columns={'repo':'repo_full'})
xtra_readme = git_xtra.readme_contents[0]


Bogdanp/awesome-advent-of-code
https://raw.githubusercontent.com/Bogdanp/awesome-advent-of-code/master/README.md


In [22]:
re_xtra = re.compile(r'\(https\://github.com/(.+?)\)')
xtra_check = re_xtra.findall(xtra_readme)
xtra_check = [i.split('/')[0] + '/' + i.split('/')[1] for i in xtra_check]
xtra_check

['dave-burke/advent-of-code-java-starter',
 'gobanos/cargo-aoc',
 'hughjdavey/aoc-kotlin-starter',
 'kindermoumoute/adventofcode',
 'staylorwr/elixir_aoc',
 'mhanberg/advent-of-code-elixir-starter',
 'AlexeSimon/adventofcode',
 'sindrekjr/AdventOfCodeBase',
 'mhanberg/advent-of-code-clojure-starter',
 'caderek/aoc-starter-js',
 'caderek/aoc-starter-ts',
 'GreenLightning/advent-of-code-downloader',
 'keirua/aoc-cli',
 'eduherminio/AoCHelper',
 'jamestomasino/advent-of-code-2019',
 'coder5506/advent-of-code-2019',
 'AnkurSheel/AdventOfCode2019',
 'alexchro93/AdventOfCode',
 'eduherminio/AoC2019',
 'encse/adventofcode',
 'sanraith/aoc2019',
 'sindrekjr/AdventOfCode',
 'viceroypenguin/adventofcode',
 'TheRealMolen/adventofcode2019',
 'voivoid/advent-of-code',
 'agrison/advent-of-code-2019',
 'alexparlett/advent-of-code-2019',
 'erdos/advent-of-code-2019',
 'fctorial/adventofcode2019-clojure',
 'fdlk/advent-2019',
 'felipecortez/advent-of-code',
 'jdlambert/advent-of-code-2019',
 'mastercak

In [23]:
repo_list.extend(xtra_check)
repo_list = list(set(repo_list))
for repo in remove_repos:
    if repo in repo_list:
        repo_list.remove(repo)
repo_list.sort(key=str.lower)
print(len(repo_list))
repo_list

353


['0legg/adventofcode',
 '0x8b/advent-of-code-2019',
 '1Computer1/advent',
 'a-red-christmas/aoc2019-ae',
 'adam-gruer/aoc2019',
 'adriennetacke/advent-of-code-2018',
 'adriennetacke/advent-of-code-2019',
 'adventofcode/2015solutions',
 'afonsojramos/advent-of-code-2018',
 'agrison/advent-of-code-2019',
 'akulumbeg/adventofcode',
 'Akumatic/Advent-of-Code',
 'albertobastos/advent-of-code-2018-nodejs',
 'AlexAegis/advent-of-code',
 'alexchro93/AdventOfCode',
 'AlexeSimon/adventofcode',
 'alexparlett/advent-of-code-2019',
 'alyti/aoc-2019',
 'AmauryCarrade/AdventOfCode2019',
 'amochtar/aoc-ranking',
 'AnkurSheel/AdventOfCode2019',
 'aran112000/Advent-of-Code-2019-PHP',
 'armstnp/advent-of-code-2019',
 'arnauddrain/advent-of-code-2019',
 'aspittel/advent-of-code',
 'atme/advent-of-code',
 'aturley/advent-of-code-2018',
 'Awjin/advent-of-code',
 'badouralix/advent-of-code-2018',
 'BartMassey/advent-of-code-2015',
 'BartMassey/advent-of-code-2016',
 'BartoszMilewski/AdventOfCode18',
 'Bartos

In [24]:
filepath = "datafiles/data.json"
zaq.scrape_github_data(repos=repo_list, filepath=filepath)


0legg/adventofcode
None
0x8b/advent-of-code-2019
https://raw.githubusercontent.com/0x8b/advent-of-code-2019/master/README.md
1Computer1/advent
https://raw.githubusercontent.com/1Computer1/advent/master/README.md
a-red-christmas/aoc2019-ae
None
adam-gruer/aoc2019
None
adriennetacke/advent-of-code-2018
https://raw.githubusercontent.com/adriennetacke/advent-of-code-2018/master/README.md
adriennetacke/advent-of-code-2019
https://raw.githubusercontent.com/adriennetacke/advent-of-code-2019/master/README.md
adventofcode/2015solutions
https://raw.githubusercontent.com/adventofcode/2015solutions/master/README.md
afonsojramos/advent-of-code-2018
https://raw.githubusercontent.com/afonsojramos/advent-of-code-2018/master/README.md
agrison/advent-of-code-2019
https://raw.githubusercontent.com/agrison/advent-of-code-2019/master/README.md
akulumbeg/adventofcode
https://raw.githubusercontent.com/akulumbeg/adventofcode/master/README.md
Akumatic/Advent-of-Code
https://raw.githubusercontent.com/Akumatic/A

derNiklaas/Advent-of-Code-2019
https://raw.githubusercontent.com/derNiklaas/Advent-of-Code/master/README.md
devries/advent_of_code_2019
https://raw.githubusercontent.com/devries/advent_of_code_2019/master/README.md
dgarroDC/advent-of-code-2019-ldpl
https://raw.githubusercontent.com/dgarroDC/advent-of-code-2019-ldpl/master/README.md
Diggsey/aoc2018
None
dlew/aoc-2016
https://raw.githubusercontent.com/dlew/aoc-2016/master/README.md
dlew/aoc-2017
https://raw.githubusercontent.com/dlew/aoc-2017/master/README.md
dmies/adventOfCode
https://raw.githubusercontent.com/dmies/adventOfCode/master/README.md
dmjio/AdventOfCode2018
https://raw.githubusercontent.com/dmjio/AdventOfCode2018/master/README.md
dmjio/aoc2017
https://raw.githubusercontent.com/dmjio/aoc2017/master/README.md
dstockto/AdventOfCode2015
https://raw.githubusercontent.com/dstockto/AdventOfCode2015/master/README.md
dunyakirkali/aoc
https://raw.githubusercontent.com/dunyakirkali/aoc/master/README.md
dzikoysk/advent-of-code
https://ra

JesperDramsch/advent-of-code
https://raw.githubusercontent.com/JesperDramsch/advent-of-code/master/README.md
jgoerner/aoc-2019
https://raw.githubusercontent.com/jgoerner/aoc-2019/master/Readme.md
jitwit/aoc
https://raw.githubusercontent.com/jitwit/aoc/master/README
jkoenig134/AdventOfCode-2019
https://raw.githubusercontent.com/jkoenig134/AdventOfCode-2019/master/README.md
joelgrus/advent2017
https://raw.githubusercontent.com/joelgrus/advent2017/master/README.md
joelgrus/advent2018
https://raw.githubusercontent.com/joelgrus/advent2018/master/README.md
joelgrus/advent2019
https://raw.githubusercontent.com/joelgrus/advent2019/master/README.md
jorispz/aoc-2019
https://raw.githubusercontent.com/jorispz/aoc-2019/master/README.md
jrfondren/adventofcode
https://raw.githubusercontent.com/jrfondren/adventofcode/master/README.md
juffalow/advent-of-code
https://raw.githubusercontent.com/juffalow/advent-of-code/master/README.md
julemand101/AdventOfCode2019
None
juliendelplanque/AdventOfCode2017With

https://raw.githubusercontent.com/mstksg/advent-of-code-api/master/README.md
mvaldesdeleon/aoc18
https://raw.githubusercontent.com/mvaldesdeleon/aoc18/master/README.md
narimiran/AdventOfCode2017
https://raw.githubusercontent.com/narimiran/AdventOfCode2017/master/README.md
narimiran/AdventOfCode2018
https://raw.githubusercontent.com/narimiran/AdventOfCode2018/master/README.md
narnolddd/adventOfCode
https://raw.githubusercontent.com/narnolddd/adventOfCode/master/README.md
natemago/adventofcode2016
None
nenadom/AdventOfCode
https://raw.githubusercontent.com/nenadom/AdventOfCode/master/README.md
nickbclifford/advent-of-code-2018
https://raw.githubusercontent.com/nickbclifford/advent-of-code-2018/master/README.md
nicuveo/advent-of-code
https://raw.githubusercontent.com/nicuveo/advent-of-code/master/README.md
NiXXeD/adventofcode
https://raw.githubusercontent.com/NiXXeD/adventofcode/master/README.md
njoubert/RaspberryPiChristmasCodingAdventure
https://raw.githubusercontent.com/njoubert/Raspbe

xSke/aoc15
None
yamnikov-oleg/adventofcode2017
https://raw.githubusercontent.com/yamnikov-oleg/adventofcode2017/master/README.md
zenieldanaku/AdventOfCode
https://raw.githubusercontent.com/zenieldanaku/AdventOfCode/master/README.md
zero-to-mastery/coding_challenge-12
https://raw.githubusercontent.com/zero-to-mastery/coding_challenge-12/master/README.md
zero-to-mastery/coding_challenge-24
https://raw.githubusercontent.com/zero-to-mastery/coding_challenge-24/master/README.md


In [25]:
git_json = pd.read_json(filepath).rename(columns={'repo':'repo_full'})
git_json=git_json[git_json.repo_full.isin(remove_repos)==False].dropna().reset_index().drop(columns='index')
git_json['author'] = git_json.repo_full.apply(lambda x: x.split('/')[0])
git_json['repo'] = git_json.repo_full.apply(lambda x: x.split('/')[-1])
display(git_json.head())
git_json.info()

,language,language_all,readme_contents,repo_full,author,repo
0,Python,{'Python': 41534},# 0x8b's [Advent of Code 2019](https://advento...,0x8b/advent-of-code-2019,0x8b,advent-of-code-2019
1,Haskell,"{'Haskell': 42115, 'Python': 503}",# Advent of Code Solutions\n\nI'm probably not...,1Computer1/advent,1Computer1,advent
2,JavaScript,{'JavaScript': 23065},# advent-of-code-2018\n[![Build Status](https:...,adriennetacke/advent-of-code-2018,adriennetacke,advent-of-code-2018
3,JavaScript,{'JavaScript': 41567},# advent-of-code-2019\nAdvent of Code 2019 🎄 J...,adriennetacke/advent-of-code-2019,adriennetacke,advent-of-code-2019
4,C,"{'C': 104592, 'Java': 70741, 'Rust': 70646, 'H...",### Advent Of Code solutions\n\nGot a solution...,adventofcode/2015solutions,adventofcode,2015solutions


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 302 entries, 0 to 301
Data columns (total 6 columns):
language           302 non-null object
language_all       302 non-null object
readme_contents    302 non-null object
repo_full          302 non-null object
author             302 non-null object
repo               302 non-null object
dtypes: object(6)
memory usage: 14.2+ KB


In [26]:
git_json.repo.value_counts().head()

advent-of-code         35
advent-of-code-2019    23
adventofcode           16
AdventOfCode           16
advent-of-code-2018    11
Name: repo, dtype: int64

In [27]:
mask = ['namen' in str(x.lower()) for x in git_json.readme_contents]
idx = git_json[mask].index
for rc in git_json.repo_full.loc[idx]:
    print(rc)
for rc in git_json.readme_contents.loc[idx]:
    print(rc)

In [28]:
# repo = 'kjeliasen/AdventOfCode'
# repo = 'Harleqin/advent-of-code-2017'
# repo = 'Bogdanp/awesome-advent-of-code'
repo = 'agrison/advent-of-code-2019/'
reporesp = zaq.github_api_request(f"https://api.github.com/repos/{repo}")


In [29]:
reporesp.json()

{'message': 'Not Found',
 'documentation_url': 'https://developer.github.com/v3'}

In [30]:
git_json.language.value_counts().head()

Python        45
JavaScript    36
Haskell       29
Rust          21
Go            18
Name: language, dtype: int64

In [31]:
git_json['readme'] = git_json.readme_contents.str.lower()

git_json['readme'] = [git_json.readme[i].replace(git_json.author[i].lower(), 'username') for i in range(0,len(git_json))]
git_json['readme'] = [git_json.readme[i].replace(git_json.repo[i].lower(), 'reponame') for i in range(0,len(git_json))]

# git_json.drop(columns='readme_contents')
# print(f'has author: {git_json.name_in_readme.sum()}\nhas repo{git_json.repo_in_readme.sum()}')
git_json.head()

,language,language_all,readme_contents,repo_full,author,repo,readme
0,Python,{'Python': 41534},# 0x8b's [Advent of Code 2019](https://advento...,0x8b/advent-of-code-2019,0x8b,advent-of-code-2019,# username's [advent of code 2019](https://adv...
1,Haskell,"{'Haskell': 42115, 'Python': 503}",# Advent of Code Solutions\n\nI'm probably not...,1Computer1/advent,1Computer1,advent,# reponame of code solutions\n\ni'm probably n...
2,JavaScript,{'JavaScript': 23065},# advent-of-code-2018\n[![Build Status](https:...,adriennetacke/advent-of-code-2018,adriennetacke,advent-of-code-2018,# reponame\n[![build status](https://travis-ci...
3,JavaScript,{'JavaScript': 41567},# advent-of-code-2019\nAdvent of Code 2019 🎄 J...,adriennetacke/advent-of-code-2019,adriennetacke,advent-of-code-2019,# reponame\nadvent of code 2019 🎄 javascript s...
4,C,"{'C': 104592, 'Java': 70741, 'Rust': 70646, 'H...",### Advent Of Code solutions\n\nGot a solution...,adventofcode/2015solutions,adventofcode,2015solutions,### advent of code solutions\n\ngot a solution...


In [32]:
git_json.drop(columns=['readme_contents']).to_json('datafiles/outdata.json', orient = 'columns')

In [33]:
json_chk = pd.read_json('datafiles/outdata.json')
display(json_chk.head())
display(json_chk.info())

,language,language_all,repo_full,author,repo,readme
0,Python,{'Python': 41534},0x8b/advent-of-code-2019,0x8b,advent-of-code-2019,# username's [advent of code 2019](https://adv...
1,Haskell,"{'Haskell': 42115, 'Python': 503}",1Computer1/advent,1Computer1,advent,# reponame of code solutions\n\ni'm probably n...
10,TypeScript,"{'TypeScript': 178903, 'Rust': 68139, 'Smarty'...",AlexAegis/advent-of-code,AlexAegis,advent-of-code,# [advent of code](https://adventofcode.com/)\...
100,Python,"{'Python': 64874, 'Julia': 27162}",ferrolho/advent-of-code,ferrolho,advent-of-code,# advent of code\n\n:santa: :christmas_tree: :...
101,Swift,{'Swift': 69603},fguchelaar/AdventOfCode2019,fguchelaar,AdventOfCode2019,# advent of code 2019\n\nmy solutions for the ...


<class 'pandas.core.frame.DataFrame'>
Int64Index: 302 entries, 0 to 99
Data columns (total 6 columns):
language        302 non-null object
language_all    302 non-null object
repo_full       302 non-null object
author          302 non-null object
repo            302 non-null object
readme          302 non-null object
dtypes: object(6)
memory usage: 16.5+ KB


None